In [2]:
import pandas as pd 
import numpy  as np
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv("dataset_cleaned.csv")
df

,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,...,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate
0,INV10002,CUST8861,SRV002,2023-12-17,Paid,No Leakage,Sale,Credit Card,Tea Powder (250g),Standard Delivery,...,0,Branch B,CASH011,SUP143,16:09:25,Premium,In-Store,No Anomaly,10002,0
1,INV10008,CUST6820,SRV003,2024-05-23,Failed,No Leakage,Sale,Net Banking,Chips (100g),Express Delivery,...,0,Branch B,CASH009,SUP166,19:20:46,Regular,Phone,No Anomaly,10008,0
2,INV10009,CUST4895,SRV001,2023-01-13,Paid,Anomaly,Return,UPI,Bananas (1kg),No Service,...,0,Online,CASH011,SUP109,18:52:28,Regular,Online,Excess Payment,10009,0
3,INV10014,CUST8666,SRV003,2023-11-08,Failed,No Leakage,Sale,UPI,Dishwash Soap (500ml),Express Delivery,...,0,Online,CASH003,SUP156,14:10:44,Wholesale,In-Store,No Anomaly,10014,0
4,INV10019,CUST7397,SRV002,2024-02-12,Paid,No Leakage,Return,Debit Card,Lentils (1kg),Standard Delivery,...,0,Online,CASH013,SUP160,19:03:48,Wholesale,Phone,No Anomaly,10019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,INV19985,CUST8806,SRV001,2024-03-27,Pending,No Leakage,Sale,Net Banking,Sugar (1kg),No Service,...,0,Branch B,CASH013,SUP121,05:01:15,Premium,Phone,No Anomaly,19985,0
3473,INV19986,CUST8426,SRV003,2023-09-06,Pending,No Leakage,Sale,Net Banking,Butter (100g),Express Delivery,...,0,Main Store,CASH009,SUP183,07:07:08,New,Phone,No Anomaly,19986,0
3474,INV19988,CUST8413,SRV002,2023-08-28,Pending,No Leakage,Sale,Net Banking,Wheat Flour (1kg),Standard Delivery,...,0,Online,CASH004,SUP100,00:33:28,New,Online,No Anomaly,19988,0
3475,INV19990,CUST8136,SRV003,2023-07-31,Paid,Anomaly,Return,Debit Card,Cooking Oil (1L),Express Delivery,...,0,Branch A,CASH013,SUP113,16:39:52,Wholesale,In-Store,Excess Payment,19990,0


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# --- Step 1: Data Loading and Preprocessing ---
# Load the original dataset
df = pd.read_csv('revenue_leak_balanced.csv')

# Handle missing values
df['Service_Category'].fillna('No Service', inplace=True)
df['Anomaly_Type'].fillna('No Anomaly', inplace=True)

# Identify categorical columns for one-hot encoding
categorical_cols = [
    'Payment_Status', 'Transaction_Type', 'Mode_of_Payment', 'Product_Name',
    'Service', 'Product_Category', 'Service_Category', 'Store_Branch',
    'Cashier_ID', 'Supplier_ID', 'Customer_Type', 'Order_Channel', 'Anomaly_Type'
]
df_preprocessed = pd.get_dummies(df, columns=categorical_cols, dtype=int)

# Drop non-predictive identifier and time-based columns
columns_to_drop = [
    'Invoice_Number', 'Customer_ID', 'Service_ID', 'Billing_Date', 'Billing_Time'
]
df_preprocessed = df_preprocessed.drop(columns=columns_to_drop, errors='ignore')

# Convert the target variable 'Leakage_Flag' to a binary numerical format
df_preprocessed['Leakage_Flag'] = df_preprocessed['Leakage_Flag'].replace({'No Leakage': 0, 'Anomaly': 1})

# --- Step 2: Split Data ---
# Separate features (X) and target (y)
X = df_preprocessed.drop('Leakage_Flag', axis=1)
y = df_preprocessed['Leakage_Flag']

# Split the data into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 3: Train the Model ---
# Initialize and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Step 4: Make Predictions and Generate Report ---
# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)

# Generate and print the classification report which includes Recall and F1-Score
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       455
           1       1.00      1.00      1.00       292

    accuracy                           1.00       747
   macro avg       1.00      1.00      1.00       747
weighted avg       1.00      1.00      1.00       747



In [8]:
df.dtypes

Invoice_Number       object
Customer_ID          object
Service_ID           object
Billing_Date         object
Payment_Status       object
Leakage_Flag         object
Transaction_Type     object
Mode_of_Payment      object
Product_Name         object
Service              object
Product_Category     object
Service_Category     object
Product_Quantity      int64
Tax_Amount          float64
Actual_Amount       float64
Billed_Amount       float64
Paid_Amount         float64
Balance_Amount      float64
Unit_Price          float64
Tax_Rate            float64
Service_Charge        int64
Discount_Amount       int64
Store_Branch         object
Cashier_ID           object
Supplier_ID          object
Billing_Time         object
Customer_Type        object
Order_Channel        object
Anomaly_Type         object
Invoice_Num_Int       int32
dtype: object

In [7]:
df['Invoice_Num_Int'] = df['Invoice_Number'].str.replace("INV", "").astype(int)

In [9]:
# Step 1: Create numeric invoice column
#df['Invoice_Num_Int'] = df['Invoice_Number'].str.replace("INV", "").astype(int)

# Step 2: Sort by invoice number
df = df.sort_values(by='Invoice_Num_Int').reset_index(drop=True)

# Step 3: Create Is_Duplicate flag (check previous/next after sorting)
df['Is_Duplicate'] = (
    (df['Invoice_Number'] == df['Invoice_Number'].shift(1)) | 
    (df['Invoice_Number'] == df['Invoice_Number'].shift(-1))
).astype(int)


In [10]:
df.head()

,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,...,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate
0,INV10002,CUST8861,SRV002,2023-12-17,Paid,No Leakage,Sale,Credit Card,Tea Powder (250g),Standard Delivery,...,0,Branch B,CASH011,SUP143,16:09:25,Premium,In-Store,No Anomaly,10002,0
1,INV10008,CUST6820,SRV003,2024-05-23,Failed,No Leakage,Sale,Net Banking,Chips (100g),Express Delivery,...,0,Branch B,CASH009,SUP166,19:20:46,Regular,Phone,No Anomaly,10008,0
2,INV10009,CUST4895,SRV001,2023-01-13,Paid,Anomaly,Return,UPI,Bananas (1kg),No Service,...,0,Online,CASH011,SUP109,18:52:28,Regular,Online,Excess Payment,10009,0
3,INV10014,CUST8666,SRV003,2023-11-08,Failed,No Leakage,Sale,UPI,Dishwash Soap (500ml),Express Delivery,...,0,Online,CASH003,SUP156,14:10:44,Wholesale,In-Store,No Anomaly,10014,0
4,INV10019,CUST7397,SRV002,2024-02-12,Paid,No Leakage,Return,Debit Card,Lentils (1kg),Standard Delivery,...,0,Online,CASH013,SUP160,19:03:48,Wholesale,Phone,No Anomaly,10019,0


In [25]:
# Filter rows where Anomaly_Type is "Duplicate Entries"
duplicate_rows = df.loc[df['Anomaly_Type'] == "Duplicate Entries", 
                       ]

duplicate_rows


,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,...,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate
8,INV10031,CUST3570,SRV001,2023-11-24,Paid,Anomaly,Return,Credit Card,Fish (1kg),No Service,...,0,Online,CASH004,SUP115,18:39:27,New,Online,Duplicate Entries,10031,1
9,INV10031,CUST4268,SRV003,2024-02-10,Paid,Anomaly,Sale,Credit Card,Coffee Powder (200g),Express Delivery,...,0,Branch B,CASH002,SUP189,16:16:16,Premium,Online,Duplicate Entries,10031,1
32,INV10097,CUST2899,SRV002,2023-12-31,Paid,Anomaly,Sale,Net Banking,Eggs (12 pack),Standard Delivery,...,0,Main Store,CASH010,SUP152,13:06:14,Premium,In-Store,Duplicate Entries,10097,1
33,INV10097,CUST8604,SRV003,2024-08-27,Pending,Anomaly,Sale,Credit Card,Apples (1kg),Express Delivery,...,0,Branch B,CASH012,SUP160,10:14:47,Regular,Phone,Duplicate Entries,10097,1
62,INV10169,CUST2363,SRV001,2023-06-05,Pending,Anomaly,Return,Cheque,Cheese (200g),No Service,...,0,Main Store,CASH012,SUP181,09:45:04,Premium,Online,Duplicate Entries,10169,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,INV19742,CUST5587,SRV002,2024-03-15,Paid,Anomaly,Sale,Cheque,Dishwash Soap (500ml),Standard Delivery,...,0,Branch B,CASH006,SUP173,16:46:32,New,Phone,Duplicate Entries,19742,1
3631,INV19742,CUST6179,SRV001,2023-06-10,Failed,Anomaly,Return,Cheque,Apples (1kg),No Service,...,0,Main Store,CASH010,SUP197,20:15:04,New,Phone,Duplicate Entries,19742,1
3636,INV19752,CUST5473,SRV002,2024-01-02,Failed,Anomaly,Return,Debit Card,Onions (1kg),Standard Delivery,...,0,Branch B,CASH015,SUP147,01:19:50,Wholesale,Online,Duplicate Entries,19752,1
3637,INV19752,CUST4850,SRV001,2024-05-28,Paid,Anomaly,Return,Debit Card,Apples (1kg),No Service,...,0,Branch A,CASH020,SUP119,16:55:27,Premium,Phone,Duplicate Entries,19752,1


In [27]:
df.shape

(3477, 31)

In [26]:
duplicate_rows['Is_Duplicate'].value_counts()


Is_Duplicate
1    155
Name: count, dtype: int64

In [22]:
# Remove rows where Is_Duplicate = 0 and Anomaly_Type = "Duplicate Entries"
df = df.drop(df[(df['Is_Duplicate'] == 0) & 
                (df['Anomaly_Type'] == "Duplicate Entries")].index)


In [28]:
df.to_csv("dataset_cleaned.csv", index=False)